<a href="https://colab.research.google.com/github/BrittonWinterrose/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/module3-introduction-to-bayesian-inference/LS_DS_143_Introduction_to_Bayesian_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 143

## Introduction to Bayesian Inference

!['Detector! What would the Bayesian statistician say if I asked him whether the--' [roll] 'I AM A NEUTRINO DETECTOR, NOT A LABYRINTH GUARD. SERIOUSLY, DID YOUR BRAIN FALL OUT?' [roll] '... yes.'](https://imgs.xkcd.com/comics/frequentists_vs_bayesians.png)

*[XKCD 1132](https://www.xkcd.com/1132/)*


## Prepare - Bayes' Theorem and the Bayesian mindset

Bayes' theorem possesses a near-mythical quality - a bit of math that somehow magically evaluates a situation. But this mythicalness has more to do with its reputation and advanced applications than the actual core of it - deriving it is actually remarkably straightforward.

### The Law of Total Probability

By definition, the total probability of all outcomes (events) if some variable (event space) $A$ is 1. That is:

$$P(A) = \sum_n P(A_n) = 1$$

The law of total probability takes this further, considering two variables ($A$ and $B$) and relating their marginal probabilities (their likelihoods considered independently, without reference to one another) and their conditional probabilities (their likelihoods considered jointly). A marginal probability is simply notated as e.g. $P(A)$, while a conditional probability is notated $P(A|B)$, which reads "probability of $A$ *given* $B$".

The law of total probability states:

$$P(A) = \sum_n P(A | B_n) P(B_n)$$

In words - the total probability of $A$ is equal to the sum of the conditional probability of $A$ on any given event $B_n$ times the probability of that event $B_n$, and summed over all possible events in $B$.

### The Law of Conditional Probability

What's the probability of something conditioned on something else? To determine this we have to go back to set theory and think about the intersection of sets:

The formula for actual calculation:

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

We can see how this relates back to the law of total probability - multiply both sides by $P(B)$ and you get $P(A|B)P(B) = P(A \cap B)$ - replaced back into the law of total probability we get $P(A) = \sum_n P(A \cap B_n)$.

This may not seem like an improvement at first, but try to relate it back to the above picture - if you think of sets as physical objects, we're saying that the total probability of $A$ given $B$ is all the little pieces of it intersected with $B$, added together. The conditional probability is then just that again, but divided by the probability of $B$ itself happening in the first place.

### Bayes Theorem

Here is is, the seemingly magic tool:

$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

In words - the probability of $A$ conditioned on $B$ is the probability of $B$ conditioned on $A$, times the probability of $A$ and divided by the probability of $B$. These unconditioned probabilities are referred to as "prior beliefs", and the conditioned probabilities as "updated."

Why is this important? Scroll back up to the XKCD example - the Bayesian statistician draws a less absurd conclusion because their prior belief in the likelihood that the sun will go nova is extremely low. So, even when updated based on evidence from a detector that is $35/36 = 0.972$ accurate, the prior belief doesn't shift enough to change their overall opinion.

There's many examples of Bayes' theorem - one less absurd example is to apply to [breathalyzer tests](https://www.bayestheorem.net/breathalyzer-example/). You may think that a breathalyzer test that is 100% accurate for true positives (detecting somebody who is drunk) is pretty good, but what if it also has 8% false positives (indicating somebody is drunk when they're not)? And furthermore, the rate of drunk driving (and thus our prior belief)  is 1/1000.

What is the likelihood somebody really is drunk if they test positive? Some may guess it's 92% - the difference between the true positives and the false positives. But we have a prior belief of the background/true rate of drunk driving. Sounds like a job for Bayes' theorem!

$$
\begin{aligned}
P(Drunk | Positive) &= \frac{P(Positive | Drunk)P(Drunk)}{P(Positive)} \\
&= \frac{1 \times 0.001}{0.08} \\
&= 0.0125
\end{aligned}
$$

In other words, the likelihood that somebody is drunk given they tested positive with a breathalyzer in this situation is only 1.25% - probably much lower than you'd guess. This is why, in practice, it's important to have a repeated test to confirm (the probability of two false positives in a row is $0.08 * 0.08 = 0.0064$, much lower), and Bayes' theorem has been relevant in court cases where proper consideration of evidence was important.

## Live Lecture - Deriving Bayes' Theorem, Calculating Bayesian Confidence

Notice that $P(A|B)$ appears in the above laws - in Bayesian terms, this is the belief in $A$ updated for the evidence $B$. So all we need to do is solve for this term to derive Bayes' theorem. Let's do it together!

$x = 2$ is an inline equation.

$$
x = 2
$$

is a block equation.

$$
\begin{aligned}
x &= 2 \\
&= 1 + 1
\end{aligned}
$$

Now let's derive Bayes!

$$
\begin{aligned}
P(A \cap B) &= P(B \cap A) \\
\\
P(A|B) &= \frac{P(A \cap B)}{P(B)} \\
\Rightarrow P(A|B)P(B) &= P(A \cap B) \\
P(B|A) &= \frac{P(B \cap A)}{P(A)} \\
\Rightarrow P(B|A)P(A) &= P(B \cap A) = P(A \cap B) \\
\Rightarrow P(A|B)P(B) &= P(B|A)P(A) \\
\Rightarrow P(A|B)&= \frac{P(B|A)P(A)}{P(B)}
\end{aligned}
$$

In [1]:
# Activity 2 - Use SciPy to calculate Bayesian confidence intervals
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bayes_mvs.html#scipy.stats.bayes_mvs

from scipy import stats
import numpy as np

coinflips = np.random.binomial(n=1, p=0.5, size=100)
print(coinflips)

[1 1 1 1 1 1 1 1 0 1 0 0 1 1 0 1 0 0 0 1 1 1 0 1 0 1 1 1 1 0 0 0 0 1 0 0 0
 1 1 0 1 0 1 0 0 1 1 0 1 0 0 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 0 0 0 1 1 1 0 1
 0 0 0 0 0 0 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 0 0 1 1]


In [2]:
# Frequentist approach (from yesterday)
def confidence_interval(data, confidence=0.95):
  """
  Calculate a confidence interval around a sample mean for given data.
  Using t-distribution and two-tailed test, default 95% confidence. 
  
  Arguments:
    data - iterable (list or numpy array) of sample observations
    confidence - level of confidence for the interval
  
  Returns:
    tuple of (mean, lower bound, upper bound)
  """
  data = np.array(data)
  mean = np.mean(data)
  n = len(data)
  stderr = stats.sem(data)
  interval = stderr * stats.t.ppf((1 + confidence) / 2., n - 1)
  return (mean, mean - interval, mean + interval)

confidence_interval(coinflips)

(0.53, 0.43046898750173684, 0.6295310124982633)

In [3]:
import pandas as pd
pd.DataFrame(coinflips).describe()

,0
count,100.000000
mean,0.530000
std,0.501614
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [4]:
stats.bayes_mvs(coinflips)

(Mean(statistic=0.53, minmax=(0.4467125284491557, 0.6132874715508443)),
 Variance(statistic=0.25680412371134015, minmax=(0.20215017434095595, 0.323311952657888)),
 Std_dev(statistic=0.5054540733507159, minmax=(0.44961113680708126, 0.5686052696360524)))

In [5]:
# Let's do something else medical
import random

# We have two groups of people, one treated one non-treated
# Treated people recover with probability 0.65
# Non-treated people recover with probability 0.4
treatment_group = np.random.binomial(n=1, p=0.65, size=40)
nontreated_group = np.random.binomial(n=1, p=0.4, size=40)

print(treatment_group)

[1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 0 0 1 0 1 1 1 0 1 1 1 0 0 1 0 0 0 1 1 1 0 0
 1 0 1]


In [6]:
import pandas as pd
df = pd.DataFrame({'treated': treatment_group,
                   'untreated': nontreated_group})
df.describe()

,treated,untreated
count,40.000000,40.000000
mean,0.575000,0.325000
std,0.500641,0.474342
min,0.000000,0.000000
25%,0.000000,0.000000
50%,1.000000,0.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [7]:
df.head()

,treated,untreated
0,1,1
1,1,1
2,1,0
3,1,1
4,1,1


In [8]:
# Frequentist hypothesis test
from scipy import stats
stats.ttest_ind(df.treated, df.untreated)

Ttest_indResult(statistic=2.29261088936325, pvalue=0.02456598260812098)

In [9]:
stats.bayes_mvs(df.treated)

(Mean(statistic=0.575, minmax=(0.44162826495682483, 0.708371735043175)),
 Variance(statistic=0.2641891891891892, minmax=(0.17912041346705607, 0.38041842711842055)),
 Std_dev(statistic=0.5105325725242656, minmax=(0.423226196574664, 0.6167806961298485)))

In [10]:
stats.bayes_mvs(df.untreated)

(Mean(statistic=0.325, minmax=(0.198634366037695, 0.451365633962305)),
 Variance(statistic=0.23716216216216218, minmax=(0.1607960744934442, 0.3415009409681985)),
 Std_dev(statistic=0.48371397556274476, minmax=(0.4009938584235975, 0.5843808184464977)))

In [0]:
# Suggested task - write your own Bayes test function
# that compares CIs from stats.bayes_mvs


## Assignment - Code it up!

Most of the above was pure math - write Python code to reproduce the results. This is purposefully open ended - you'll have to think about how you should represent probabilities and events. You can and should look things up, and as a stretch goal - refactor your code into helpful reusable functions!

If you're unsure where to start, check out [this blog post of Bayes theorem with Python](https://dataconomy.com/2015/02/introduction-to-bayes-theorem-with-python/) - you could and should create something similar!

Stretch goal - apply a Bayesian technique to a problem you previously worked (in an assignment or project work) on from a frequentist (standard) perspective.

### Yesterday (importing the med dataset)

In [12]:
# Getting started with drug data
# http://archive.ics.uci.edu/ml/datasets/Drug+Review+Dataset+%28Drugs.com%29

!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip

--2018-12-06 18:05:22--  http://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42989872 (41M) [application/zip]
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip    100%[===================>]  41.00M  14.4MB/s    in 2.9s    

2018-12-06 18:05:25 (14.4 MB/s) - ‘drugsCom_raw.zip’ saved [42989872/42989872]



In [13]:
!unzip drugsCom_raw.zip

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [14]:
!head drugsComTrain_raw.tsv

	drugName	condition	review	rating	date	usefulCount
206461	Valsartan	Left Ventricular Dysfunction	"""It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"""	9.0	May 20, 2012	27
95260	Guanfacine	ADHD	"""My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. 
We have tried many different medications and so far this is the most effective."""	8.0	April 27, 2010	192
92703	Lyb

In [15]:
df = pd.read_table('drugsComTrain_raw.tsv')
df.head()

### FROM YESTERDAY 
"""
I want to take the df, filter by condition, drug, confidence interval, sample size cutoff)
Then loop through all the drugs for a specific condition and calculate their
mean, top limit, and bottom limit. 
"""
# Create Confidence Interval Function
def confidence_interval (data, ci_percent):
  data = np.array(data) # Makes sure our data is in a numpy array
  mean = np.mean(data)
  n = len(data)
  stderr = stats.sem(data)
  interval = stderr * stats.t.ppf((1 + ci_percent) / 2., n - 1)
  return (mean, mean - interval, mean + interval)


def condition_compare (df, condition_id, ci_percent, sample_size_cutoff):
  output_names = ["Drug Name", "Sample Mean", "Lower Bound", "Upper Bound", "Sample Size"]
  drug_compare = []
  data = df[df.condition == condition_id]
  for drug in data.drugName.unique():
    one_drug = data[data.drugName == drug].rating
    if one_drug.size > sample_size_cutoff:
#
      mean, ilower, iupper= confidence_interval(one_drug, ci_percent)
      entry = [drug, mean, ilower, iupper, one_drug.size]
      drug_compare.append(entry)
  return pd.DataFrame(drug_compare, columns=output_names)


df2 = condition_compare(df, "Depression", 0.95, 10).sort_values(by="Sample Mean", ascending=False)
df2.head(5)

,Drug Name,Sample Mean,Lower Bound,Upper Bound,Sample Size
62,Niacin,9.857143,9.647474,10.066812,14
47,Tramadol,9.288462,8.934000,9.642923,52
68,Clomipramine,9.181818,8.106160,10.257476,11
37,Xanax,9.166667,8.603654,9.729679,42
17,Methylphenidate,9.160000,8.789263,9.530737,25


### Some stuff


In [0]:
# https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
import re

In [20]:
df_train = pd.read_table('drugsComTrain_raw.tsv')
df_test = pd.read_table('drugsComTest_raw.tsv')

df_main = pd.concat([df_train, df_test], axis=0)
df_main

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37
5,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2.0,"November 28, 2015",43
6,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1.0,"March 7, 2017",5
7,102654,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",10.0,"March 14, 2015",32
8,74811,Keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",1.0,"August 9, 2016",11
9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8.0,"December 8, 2016",1


In [21]:
# Clean up text
pd.set_option('display.width', 1000)
rx_pat = r"(\\r)|(\\n)|(\\t)|(\\f)|(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(&#039;)|(\d\s)|(\d)|(\/)"
rx_pat_wSpace = r"(\-)|(\\)|(\s{2,})"
    
df_main['review'].replace(regex=True,inplace=True,to_replace=rx_pat, value=r'')
df_main['review'].replace(regex=True,inplace=True,to_replace=rx_pat_wSpace, value=r' ')
df_main.review

0        It has no side effect I take it in combination...
1        My son is halfway through his fourth week of I...
2        I used to take another oral contraceptive whic...
3        This is my first time using any form of birth ...
4        Suboxone has completely turned my life around ...
5        nd day on mg started to work with rock hard er...
6        He pulled out but he cummed a bit in me I took...
7        Abilify changed my life There is hope I was on...
8         I Ve had nothing but problems with the Kepper...
9        I had been on the pill for many years When my ...
10       I have been on this medication almost two week...
11       I have taken anti depressants for years with s...
12       I had Crohns with a resection years ago and ha...
13       Have a little bit of a lingering cough from a ...
14       Started Nexplanon months ago because I have a ...
15       I have been taking Saxenda since July I had se...
16       This drug worked very well for me and cleared .

In [22]:
# Nailed it. 
df_main['review'] = df_main['review'].str.lower()

df_main['review']

0        it has no side effect i take it in combination...
1        my son is halfway through his fourth week of i...
2        i used to take another oral contraceptive whic...
3        this is my first time using any form of birth ...
4        suboxone has completely turned my life around ...
5        nd day on mg started to work with rock hard er...
6        he pulled out but he cummed a bit in me i took...
7        abilify changed my life there is hope i was on...
8         i ve had nothing but problems with the kepper...
9        i had been on the pill for many years when my ...
10       i have been on this medication almost two week...
11       i have taken anti depressants for years with s...
12       i had crohns with a resection years ago and ha...
13       have a little bit of a lingering cough from a ...
14       started nexplanon months ago because i have a ...
15       i have been taking saxenda since july i had se...
16       this drug worked very well for me and cleared .

In [0]:
# VECTORIZE IT (One Hot Encode It)
# Each word becomes one feature (column)
# This is probably where I'll get lost.
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(df_main['review'])

# Define my X & create my matrix with n things and n features
X = cv.transform(df_main['review'])

# Define my y with 
y = df_main["rating"]

In [24]:
X

<215063x71335 sparse matrix of type '<class 'numpy.int64'>'
	with 12366059 stored elements in Compressed Sparse Row format>

In [25]:
y.head()

0    9.0
1    8.0
2    5.0
3    8.0
4    9.0
Name: rating, dtype: float64

In [26]:
np.size(X,0)

215063

In [0]:
y_rank = []
for i in y:
  if i <= 4:
    y_rank.append(-1)
  elif i >= 7:
    y_rank.append(1)
  else:
    y_rank.append(0)

In [34]:
# Make sure they're the same number of n. 
y_rank = np.asarray(y_rank)
np.size(y_rank,0)

215063

In [35]:
np.random.seed()
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y_rank, train_size = 0.7)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
# Accuracy for C=0.01: 0.7125497915342768
# Accuracy for C=0.05: 0.7175405694446597
# Accuracy for C=0.25: 0.7197724701250794
# Accuracy for C=0.5: 0.7199584618484477
#   

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Accuracy for C=0.01: 0.7862955098498117
Accuracy for C=0.05: 0.7973465180799454
Accuracy for C=0.25: 0.8066306049380803
Accuracy for C=0.5: 0.811280398022288
Accuracy for C=1: 0.8187045676467397


In [36]:
# Pick a model with a C value. 
X_train, X_test, y_train, y_test = train_test_split(X, y_rank, train_size = 0.823)

final_model = LogisticRegression(C=1)
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, final_model.predict(X_test)))
# 

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Final Accuracy: 0.8208421992802165


In [37]:
# TODO
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:100]:
    print (best_positive)
print("\n")
    
#     ('excellent', 0.9288812418118644)
#     ('perfect', 0.7934641227980576)
#     ('great', 0.675040909917553)
#     ('amazing', 0.6160398142631545)
#     ('superb', 0.6063967799425831)
    
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:100]:
    print (best_negative)
    
#     ('worst', -1.367978497228895)
#     ('waste', -1.1684451288279047)
#     ('awful', -1.0277001734353677)
#     ('poorly', -0.8748317895742782)
#     ('boring', -0.8587249740682945)

('predominant', 2.346319049989232)
('bac', 2.3047457907973907)
('disappointed', 2.2743340975413635)
('marginally', 2.2731991838282277)
('combative', 2.266242753146455)
('garbage', 2.205915602330579)
('mgpill', 2.1519117768599316)
('proair', 2.1454975070309246)
('rudeness', 2.138613162011422)
('adhesion', 2.123559710988908)
('desease', 2.122890483649754)
('hugging', 2.1118793143092973)
('quack', 2.099023845920983)
('explanations', 2.0844288313880086)
('belbuca', 2.082880077499139)
('useless', 2.0658112860981537)
('clotrimazole', 2.0517817376636978)
('scam', 2.0309016619510234)
('mirvaso', 2.0260556418052675)
('minestrin', 2.0124723749779556)
('dp', 1.9906138206253536)
('fiortal', 1.9830503921245866)
('tazoratene', 1.977061553490991)
('pamphlet', 1.9769918287750758)
('quotfat', 1.9768479106378618)
('contraindications', 1.958284600137881)
('mylanta', 1.9546574980021532)
('anaphylaxis', 1.9518915278205344)
('outfits', 1.9174037487004316)
('rained', 1.9150410000595763)
('advertise', 1.91238

('temptation', 1.439320737621822)
('ubiprofen', 1.4064603559870859)
('proair', 1.3868646866296943)
('dietary', 1.3249160936587947)
('quotsex', 1.3125054629973174)
('streets', 1.312447426293259)
('indocin', 1.2734329920976153)
('lomedia', 1.1976396069227946)
('toxicity', 1.1964133946954263)
('anaphylaxis', 1.1871239411598091)
('qualitest', 1.1806090000640015)
('completion', 1.1733079901968428)
('tobe', 1.1594299313923782)
('upsetting', 1.1582521165777122)
('outweighing', 1.1579158183223566)
('trilo', 1.157247572469644)
('badits', 1.1480789935563176)
('gurgles', 1.1388097427737753)
('fealing', 1.1302635028595425)
('balling', 1.1252243286393724)
('useless', 1.119267158032575)
('overpriced', 1.096336018512172)
('swirling', 1.0956189311734361)
('absurd', 1.0923469938349888)
('tendons', 1.0884519712182004)
('acnes', 1.0803579545733635)
('quack', 1.072144984122407)
('belsomra', 1.071368642428284)
('vienna', 1.0696890913958026)
('tardive', 1.0694589404958772)
('nest', 1.0687581237859676)
('thatcaused', 1.0682715766483195)
('hairline', 1.063770417077139)
('advertising', 1.0595514913912698)
('dime', 1.0548237971765626)
('impulsivity', 1.0475831911498248)
('selfinjuroussuicidal', 1.0466844971305453)
('snaps', 1.0397026597770997)
('flashesit', 1.0330561630529767)
('gambler', 1.032835253201044)
('badit', 1.0305812441281303)
('disappointed', 1.0253486593934662)
('yazzoloft', 1.0199824972306795)
('thirties', 1.0167575456016795)
('conflicts', 1.0115850022624533)
('stems', 1.0090574354141286)
('consistantly', 1.0090404260107972)
('healthcare', 1.0089003477375007)
('cripple', 1.007072937619953)
('stinks', 1.0062732904789746)
('adversely', 1.0019328019137512)
('ineffective', 1.0015758354767306)
('pronto', 0.9997259422493763)
('prospect', 0.9961794041945937)
('dizzines', 0.9853955137295354)
('waste', 0.984828995142717)
('nightmaresteer', 0.9823125039045522)
('sputum', 0.9793929890493381)
('unnecessary', 0.9791889500291214)
('nowmg', 0.9769579501289882)
('advertise', 0.9763087437306701)
('breathless', 0.9714474183377245)
('understatement', 0.971248179041978)
('gibberish', 0.9706244175241309)
('dissatisfied', 0.9677523417328603)
('restrict', 0.9655958023444245)
('moodythe', 0.9648572685415108)
('respiclick', 0.9639541653819551)
('garbage', 0.9628603084938377)
('anxietyafter', 0.9583363119191162)
('yah', 0.9567671888744552)
('coating', 0.9484561670035363)
('fitsi', 0.9473473439568236)
('questionable', 0.9440942364711961)
('hurrah', 0.9409743633036236)
('daylight', 0.9399117047390577)
('hoarse', 0.9397434443484344)
('belbuca', 0.9379620402424741)
('thisthis', 0.9371072627473479)
('oxymorphone', 0.9366757102780021)
('pil', 0.9352572507498175)
('novemberi', 0.9315688525110285)
('higheri', 0.9305011359253039)
('perk', 0.9295736889026841)
('crampsalso', 0.9281088878860814)
('explanations', 0.9269246467868377)
('secondi', 0.9263397391838128)
('workrelated', 0.9225769882253657)
('reckless', 0.9216600292090513)
('redder', 0.9212014740302621)
('unrelenting', 0.9197614010855345)
('workschool', 0.9182816262352054)
('disclosure', 0.915189132487718)
('definition', 0.9148451042984022)
('wayy', 0.9142080219071352)
('badlyi', 0.9133303541681054)
('sulfer', 0.9131892529090855)
('pit', 0.9122615814970801)
('gabriella', 0.9117880081905312)
('aligned', 0.9110144016112635)


('handfuls', -1.179568643498087)
('enabled', -1.1578457859079594)
('chlamydia', -1.1522377920176137)
('backmy', -1.0866649697796824)
('tracks', -1.0847315264503656)
('moves', -1.0751536014441379)
('cellcept', -1.0740774439271572)
('quotbirth', -1.0703730495347479)
('downwards', -1.0648370623636652)
('driven', -1.0620226523624627)
('soul', -1.0579542505608248)
('excellent', -1.0507233783035748)
('challenging', -1.0421878293757125)
('doable', -1.0379127049465968)
('vital', -1.0292556022667108)
('peacefully', -1.0274985632071953)
('attempting', -1.0274457571057607)
('primarily', -1.0274191676691957)
('wretched', -1.021638871421151)
('exaggeration', -1.01674651902562)
('ty', -1.0128641938384582)
('tenuate', -1.0068140249302624)
('ligation', -1.0055809504916555)
('hypersensitivity', -1.0034834349682886)
('cleans', -0.9888778115694756)
('tegretol', -0.9868011926777598)
('trialed', -0.9852980890393317)
('ducolax', -0.9845086393261836)
('duodenal', -0.9806806843875522)
('crampingbut', -0.977064088287594)
('obliterated', -0.9755018176650323)
('skull', -0.9747332073229684)
('pet', -0.9737994403158644)
('complexion', -0.9648125160000877)
('recurrent', -0.961511462657212)
('triphasic', -0.9614220525817856)
('scabs', -0.9594328872226182)
('aggravate', -0.9586173976922988)
('snippy', -0.956489056835806)
('quotbreak', -0.9548584171943082)
('diluted', -0.9547638502701482)
('fib', -0.9425744029277745)
('progesterineonly', -0.9423003732522599)
('lightening', -0.9400553603142167)
('pmam', -0.9396897683899859)
('tbsp', -0.9391067964794787)
('turnaround', -0.9365886748297703)
('argue', -0.9361218962112096)
('rolls', -0.9345936296336708)
('consent', -0.934344310309122)
('addictions', -0.9280555071965295)
('monththe', -0.9259856171135837)
('trilosprintec', -0.9213080114282263)
('allows', -0.9197086905869021)
('cured', -0.9164326042754524)
('parti', -0.9157760359612807)
('snacking', -0.913330861775241)
('nodding', -0.9109063179293276)
('miracle', -0.9100255806417475)
('downfall', -0.9065978174575182)
('panel', -0.9062599574374364)
('saved', -0.9026841773409567)
('def', -0.9008721583232032)
('humiliating', -0.8984495860284347)
('vimpat', -0.8982715581002716)
('debilitated', -0.8977288077604935)
('itno', -0.8968498245446076)
('butit', -0.8887393539919856)
('complaint', -0.8866200613011979)
('tn', -0.8851767837327227)
('goodmy', -0.8845215695680272)
('trich', -0.8837548779952334)
('saviour', -0.8837530485560141)
('ivy', -0.8816538571868856)
('greatit', -0.8778065351339519)
('ef', -0.877362408047274)
('definite', -0.8749218827372844)
('pramipexole', -0.8726273306706268)
('bricks', -0.8710206006682977)
('periodit', -0.8698528182377565)
('draw', -0.8682840087125823)
('hasn', -0.8653992852333877)
('drixoral', -0.8616987183993535)
('wonderfulno', -0.8596889789590699)
('km', -0.8594687600949712)
('tecfidera', -0.8584703492985946)
('facechestback', -0.857621541019062)
('chewed', -0.8558626898520499)
('thoughthe', -0.8531178614947688)
('abscess', -0.8522862512250331)
('prehypertension', -0.8520923932738506)
('compares', -0.8507846668444728)
('sale', -0.8465903159036943)
('bust', -0.8448652910897996)
('soothes', -0.8443537515893638)
('eradicated', -0.844309112056369)
('elsethen', -0.8408745344965444)
('begining', -0.8398946145032301)
('attacksi', -0.8394239875776783)
('joking', -0.8364868292847951)
